In [18]:
import pandas as pd
import numpy as np
import torch
from torchvision.datasets import Caltech256, CIFAR100
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers

from library.taxonomy_constructors import CrossPredictionsTaxonomy
from library.models import ResNetModel
from library.datasets import Caltech256DataModule, CIFAR100DataModule

# Load dataset information
caltech256_labels = Caltech256(root="datasets/caltech256", download=False).categories
cifar100_labels = CIFAR100(root="datasets/cifar100", train=True, download=False).classes

print(f"Caltech-256 classes: {len(caltech256_labels)}")
print(f"CIFAR-100 classes: {len(cifar100_labels)}")

# Reduce the precision of matrix multiplication to speed up training
torch.set_float32_matmul_precision("medium")

Caltech-256 classes: 257
CIFAR-100 classes: 100


In [19]:
# Configuration
TRAIN = False  # Set to True to train models from scratch


def train_dataset_model(
    dataset_module, dataset_name, logger_name, model_name, num_classes, training_config
):
    """Train a ResNet model for a specific dataset"""
    tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs", name=logger_name)

    trainer = Trainer(
        max_epochs=training_config["max_epochs"],
        logger=tb_logger if TRAIN else False,
        callbacks=[
            ModelCheckpoint(
                dirpath="checkpoints",
                monitor="val_loss",
                mode="min",
                save_top_k=1,
                filename=model_name,
                enable_version_counter=False,
            )
        ],
    )

    if TRAIN:
        model = ResNetModel(
            num_classes=num_classes,
            architecture="resnet50",
            optim=training_config["optim"],
            optim_kwargs=training_config["optim_kwargs"],
            **training_config["addition"],
        )
        trainer.fit(model, datamodule=dataset_module)
        results = trainer.test(datamodule=dataset_module, ckpt_path="best")
    else:
        model = ResNetModel.load_from_checkpoint(f"checkpoints/{model_name}.ckpt")
        results = trainer.test(model, datamodule=dataset_module)

    return results

In [20]:
caltech256_dataset = Caltech256DataModule()
caltech256_config = {
    "max_epochs": 50,
    "optim": "sgd",
    "optim_kwargs": {
        "lr": 0.01,
        "momentum": 0.9,
        "weight_decay": 5e-4,
    },
    "addition": {},
}
caltech256_results = train_dataset_model(
    caltech256_dataset,
    "Caltech-256",
    "caltech256_real",
    "resnet50-caltech256-min-val-loss",
    len(caltech256_labels),
    caltech256_config,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 48/48 [00:12<00:00,  3.97it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.6928104758262634
        eval_loss           1.4151779413223267
        hp_metric           0.6928104758262634
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [21]:
cifar100_dataset = CIFAR100DataModule()
cifar100_config = {
    "max_epochs": 100,
    "optim": "adamw",
    "optim_kwargs": {
        "lr": 0.001,
        "weight_decay": 0.001,
    },
    "addition": {
        # Add learning rate scheduling
        "lr_scheduler": "multistep",
        "lr_scheduler_kwargs": {
            "milestones": [30, 60, 80],
            "gamma": 0.1,
        },
    },
}
cifar100_results = train_dataset_model(
    cifar100_dataset,
    "CIFAR-100",
    "cifar100_real",
    "resnet152-cifar100-min-val-loss",
    len(cifar100_labels),
    cifar100_config,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 157/157 [00:03<00:00, 46.11it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.5600000023841858
        eval_loss           1.6364234685897827
        hp_metric           0.5600000023841858
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [22]:
PREDICT = False

if PREDICT:
    # Load trained models
    model_caltech256 = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-caltech256-min-val-loss.ckpt"
    )
    model_caltech256.eval()

    model_cifar100 = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet152-cifar100-min-val-loss.ckpt"
    )
    model_cifar100.eval()

    trainer = Trainer(logger=False, enable_checkpointing=False)

    caltech256_on_cifar100 = trainer.predict(
        model_caltech256, datamodule=cifar100_dataset
    )

    cifar100_on_caltech256 = trainer.predict(
        model_cifar100, datamodule=caltech256_dataset
    )

    # Convert predictions to class indices
    predictions_caltech256_on_cifar100 = torch.cat(caltech256_on_cifar100).argmax(dim=1)  # type: ignore
    predictions_cifar100_on_caltech256 = torch.cat(cifar100_on_caltech256).argmax(dim=1)  # type: ignore

    # Get ground truth targets
    cifar100_targets = torch.cat(
        [label for _, label in cifar100_dataset.predict_dataloader()]
    )
    caltech256_targets = torch.cat(
        [label for _, label in caltech256_dataset.predict_dataloader()]
    )

    pd.DataFrame(
        {
            "cifar100": cifar100_targets,
            "predictions_caltech256_on_cifar100": predictions_caltech256_on_cifar100,
        }
    ).to_csv("data/cifar100_caltech256_cifar100_real_predictions.csv", index=False)
    pd.DataFrame(
        {
            "caltech256": caltech256_targets,
            "predictions_cifar100_on_caltech256": predictions_cifar100_on_caltech256,
        }
    ).to_csv("data/cifar100_caltech256_caltech256_real_predictions.csv", index=False)

# Load prediction results
df_cifar100 = pd.read_csv("data/cifar100_caltech256_cifar100_real_predictions.csv")
df_caltech256 = pd.read_csv("data/cifar100_caltech256_caltech256_real_predictions.csv")

In [23]:
# Construct taxonomy from cross-domain predictions
taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
    cross_domain_predictions=[
        (
            0,
            1,
            np.array(
                df_caltech256["predictions_cifar100_on_caltech256"], dtype=np.intp
            ),
        ),  # CIFAR-100 -> Caltech-256
        (
            1,
            0,
            np.array(df_cifar100["predictions_caltech256_on_cifar100"], dtype=np.intp),
        ),  # Caltech-256 -> CIFAR-100
    ],
    domain_targets=[
        (0, np.array(df_cifar100["cifar100"], dtype=np.intp)),
        (1, np.array(df_caltech256["caltech256"], dtype=np.intp)),
    ],
    domain_labels={0: cifar100_labels, 1: caltech256_labels},
    relationship_type="mcfp",
    threshold=0.3,
)

In [24]:
# Generate and save taxonomy visualizations
taxonomy.visualize_graph("CIFAR-100 and Caltech-256 Cross-Domain Taxonomy").save_graph(
    "output/cifar100_caltech256_real_taxonomy.html"
)

In [25]:
# Build universal taxonomy
taxonomy.build_universal_taxonomy()
taxonomy.visualize_graph("CIFAR-100 and Caltech-256 Universal Taxonomy").save_graph(
    "output/cifar100_caltech256_real_universal_taxonomy.html"
)